In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 fosforio fosforml numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 

In [5]:
!pip install dice-ml seaborn imblearn simplejson scipy xgboost pandas dice-ml refractml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn dash dtale pydantic-settings

     |████████████████████████████████| 2.5MB 4.1MB/s eta 0:00:01
     |████████████████████████████████| 296kB 80.4MB/s eta 0:00:01
     |████████████████████████████████| 143kB 79.1MB/s eta 0:00:01
     |████████████████████████████████| 34.5MB 75.9MB/s eta 0:00:01
     |████████████████████████████████| 4.5MB 74.7MB/s eta 0:00:01
     |████████████████████████████████| 12.4MB 76.0MB/s eta 0:00:01
     |████████████████████████████████| 51kB 66.8MB/s eta 0:00:01
     |████████████████████████████████| 17.3MB 73.7MB/s eta 0:00:01
     |████████████████████████████████| 11.1MB 73.2MB/s eta 0:00:01
     |████████████████████████████████| 327kB 83.6MB/s eta 0:00:01
     |████████████████████████████████| 16.4MB 74.6MB/s eta 0:00:01
     |████████████████████████████████| 9.2MB 78.7MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 76.2MB/s eta 0:00:01
     |████████████████████████████████| 7.5MB 77.1MB/s eta 0:00:01
     |████████████████████████████████| 14.4MB 74.2MB/s et

     |████████████████████████████████| 61kB 70.9MB/s eta 0:00:01
     |████████████████████████████████| 307kB 82.0MB/s eta 0:00:01
     |████████████████████████████████| 368kB 71.5MB/s eta 0:00:01
     |████████████████████████████████| 61kB 74.1MB/s eta 0:00:01
     |████████████████████████████████| 4.7MB 63.6MB/s eta 0:00:01
     |████████████████████████████████| 1.2MB 62.8MB/s eta 0:00:01
     |████████████████████████████████| 307kB 81.0MB/s eta 0:00:01
     |████████████████████████████████| 4.5MB 63.9MB/s eta 0:00:01
     |████████████████████████████████| 112kB 83.2MB/s eta 0:00:01
     |████████████████████████████████| 235kB 79.0MB/s eta 0:00:01
     |████████████████████████████████| 71kB 72.3MB/s eta 0:00:01
     |████████████████████████████████| 931kB 76.9MB/s eta 0:00:01
     |████████████████████████████████| 235kB 78.4MB/s eta 0:00:01
     |████████████████████████████████| 102kB 81.9MB/s ta 0:00:01
     |████████████████████████████████| 256kB 55.3MB/s eta 0:00:01

     |████████████████████████████████| 1.3MB 80.0MB/s 


     |████████████████████████████████| 51kB 65.5MB/s eta 0:00:01
     |████████████████████████████████| 71kB 70.6MB/s eta 0:00:01
     |████████████████████████████████| 225kB 65.9MB/s eta 0:00:01
     |████████████████████████████████| 645kB 50.6MB/s eta 0:00:01
     |████████████████████████████████| 79.9MB 66.3MB/s eta 0:00:01
     |████████████████████████████████| 491kB 78.2MB/s eta 0:00:01
     |████████████████████████████████| 983kB 75.5MB/s eta 0:00:01
     |████████████████████████████████| 430kB 74.3MB/s eta 0:00:01
     |████████████████████████████████| 358kB 73.7MB/s eta 0:00:01
     |████████████████████████████████| 61kB 73.0MB/s eta 0:00:01
     |████████████████████████████████| 51kB 67.5MB/s eta 0:00:01
     |████████████████████████████████| 112kB 83.8MB/s eta 0:00:01
     |████████████████████████████████| 3.7MB 77.7MB/s eta 0:00:01
     |████████████████████████████████| 522kB 76.6MB/s eta 0:00:01
     |████████████████████████████████| 686kB 78.1MB/s eta 0:00:0

     |████████████████████████████████| 2.8MB 76.9MB/s eta 0:00:01
     |████████████████████████████████| 5.4MB 43.6MB/s eta 0:00:01
     |████████████████████████████████| 2.1MB 57.6MB/s eta 0:00:01
     |████████████████████████████████| 43.6MB 78.2MB/s eta 0:00:01
     |████████████████████████████████| 7.0MB 82.8MB/s eta 0:00:01
Failed to build shutils dash-colorscales dash-daq htmlmin
ERROR: refractio 2.1.5.4 has requirement pandas==2.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: mlflow 2.10.0 has requirement importlib-metadata!=4.7.0,<8,>=3.7.0, but you'll have importlib-metadata 8.0.0 which is incompatible.
ERROR: mlflow 2.10.0 has requirement packaging<24, but you'll have packaging 24.1 which is incompatible.
ERROR: mlflow 2.10.0 has requirement pytz<2024, but you'll have pytz 2024.1 which is incompatible.
ERROR: fosforio 1.0.1 has requirement pandas==2.0.0, but you'll have pandas 2.0.3 which is incompatible.
ERROR: snowflake-snowpark-python 1.0.0 has require

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from imblearn.over_sampling import SMOTE
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
snowflake.get_connection(connection_name="HR Attrition conn")
df = get_dataframe("Final_HR_Attrition_Data")
df

In [ ]:
print(df.isnull().sum())

In [ ]:
df = df.dropna()

In [ ]:
print(df.isnull().sum())

In [ ]:
df = df.drop(["USER_ID", "JOB_START_DATE", "JOB_END_DATE", "SCHOOL_END_DATE"], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
CATEGORICAL_COLUMNS = ["MAPPED_ROLE_CLEAN","SEX", "ETHNICITY","HOSPITAL_TYPE", "HOSPITAL_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE", 
                       "DEGREE_CLEAN","BUSINESS_TRAVEL","ENVIRONMENT_SATISFACTION","JOB_SATISFACTION","MARTIAL_STATUS","PERFORMANCE_RATING","RELATIONSHIP_SATISFACTION","WORK_LIFE_BALANCE"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR","AGE", "OVER_TIME_HOURS", "PERCENTAGE_SALARY_HIKE",]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [ ]:
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS 

In [ ]:
feature_columns = [col for col in feature_columns if col in df.columns]
LABEL_COLUMNS = [col for col in LABEL_COLUMNS if col in df.columns]

In [ ]:
X = df[feature_columns]
Y = df[LABEL_COLUMNS]

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size = 0.20, random_state = 42)

In [ ]:
## train_df and test_df are both snowpark dataframes
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline

categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)

numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)


pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', XGBClassifier())
])

pipeline.fit(X_train, y_train)

result = pipeline.predict(X_test)

In [ ]:
result

In [ ]:
from joblib import dump, load
filename = "HR_Attrition_ml_model.joblib"
dump(pipeline, filename)

In [ ]:
import requests

In [ ]:
test = X_test
train = X_train
train["CHURN"] = train["CHURN"].apply(lambda x: 1 if x else 0)
pred = model.predict(test)
pred["probabilities"] = model.predict_proba(test)[["predict_proba_True"]]
test["PREDICTION"] = pred["PREDICTION"]
test["probabilities"] = pred["probabilities"]
test["CHURN"] = test["CHURN"].apply(lambda x: 1 if x else 0)

y_prob = np.array(test["probabilities"])
y_test = pred["CHURN"]
y_pred = pred["PREDICTION"]
X_train = train.drop(["CHURN"], axis=1)
X_test = pred.drop(["CHURN", "PREDICTION", "probabilities"], axis=1)
y_train = train["CHURN"]

In [ ]:
y_pred = pipeline.predict (X_test)
y_prob = pipeline.predict_proba(X_test)

In [ ]:
#@scoring_func
def score(model, request):
    payload_dict = request.json["payload"]
    data = pd.DataFrame(payload_dict,index=[0])
    prediction = str(model.predict(data)[0])
    return prediction

In [ ]:
## registering the model in refract.
tmp = register_model(pipeline, 
               score, 
               name="HR_Analytic_ml_model", 
               description="HR_ANALYTICS_model_trained_using _ml",
               flavour=MLModelFlavours.sklearn,
               model_type="classification",
               init_script="\\n pip install fosforml \\n pip install fosforio[snowflake] \\n pip install seaborn \\n pip install snowflake-connector-python[pandas] \\n pip install joblib==1.3.2 scikit-learn=1.3.2",
               y_true=y_test,
               y_pred=y_pred, 
               features=X_train.columns,
               labels=[0,1],
               input_type="json", 
               explain_ai=True,
               prob=y_prob,
               x_train=X_train, 
               x_test=X_test, 
               y_train=y_train,
               y_test=y_test,
               feature_names=X_train.columns.tolist(),
               original_features=X_train.columns.tolist(),
               feature_ids=X_train.columns,
               target_names=['NOT LEFT','LEFT'],
               kyd=True, kyd_score = True)


In [ ]:
payload  = {"payload": X_test.iloc[0].to_dict()}
payload